In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
nltk.download("all")
import re
from sklearn.feature_extraction.text import *
from sklearn.metrics.pairwise import *

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\amith\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\amith\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\amith\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\amith\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\amith\AppData\Roaming\nltk_data...
[

In [2]:
list_of_stopwords = nltk.corpus.stopwords.words("english")

In [3]:
def lemmatise(w):

    part_of_speech = nltk.pos_tag([w])[0][1].lower()

    if part_of_speech[0] == "n":
        pos = "n"
    elif part_of_speech[0] == "v":
        pos = "v"
    elif part_of_speech[0] == "j":
        pos = "adj"
    elif part_of_speech[0] == "r":
        pos = "adv"
    else:
        pos = part_of_speech

    return nltk.stem.WordNetLemmatizer().lemmatize(w, pos)

In [4]:
df = pd.read_csv("udemy_course_data.csv")
courses = df["course_title"].values

list_of_cleaned_courses = []

for i in courses:
    tokenised_words = nltk.tokenize.word_tokenize(i)
    p = []
    q = ""
    r = []
    for j in tokenised_words:
        p.append(nltk.stem.PorterStemmer().stem(j))
    for j in p:
        if j not in list_of_stopwords:
            q += (j + " ")
    q = q.strip()
    r = re.sub(r'[^a-zA-Z0-9\s]',"",q)
    list_of_cleaned_courses.append(r)

df_cleaned = df.copy()
df_cleaned["course_title_cleaned"] = list_of_cleaned_courses

In [5]:
tfidf = TfidfVectorizer()
vect = tfidf.fit_transform(df_cleaned["course_title_cleaned"])

cos_sim = cosine_similarity(vect)
cos_sim

array([[1.        , 0.08524937, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.08524937, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.22548828],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.08831355],
       [0.        , 0.        , 0.        , ..., 0.22548828, 0.08831355,
        1.        ]])

In [6]:
similarity_matrix = []
similarity_matrix_indices = []
for i in cos_sim:
    cos = pd.Series(i)
    coz = cos.sort_values(ascending=False)
    pk = coz.index.values[1:]
    similarity_matrix_indices.append(pk)

In [16]:
df_cleaned["Similarity Indices"] = similarity_matrix_indices
df_cleaned.head()

,course_id,course_title,url,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,published_timestamp,subject,profit,published_date,published_time,year,month,day,course_title_cleaned,Similarity Indices
0,1070968,Ultimate Investment Banking Course,https://www.udemy.com/ultimate-investment-banking-course/,True,200,2147,23,51,All Levels,1.5 hours,2017-01-18T20:58:58Z,Business Finance,429400,2017-01-18,20:58:58Z,2017,1,18,ultim invest bank cours,"[39, 242, 3479, 2719, 659, 419, 2807, 1215, 3008, 26, 1073, 420, 2648, 565, 229, 2505, 724, 139, 335, 2626, 3015, 3662, 1992, 3238, 172, 2708, 766, 173, 1849, 530, 364, 3529, 1006, 1216, 417, 466, 446, 992, 441, 127, 271, 538, 567, 892, 2645, 73, 177, 1152, 539, 87, 773, 775, 365, 105, 497, 338, 765, 2317, 805, 847, 1127, 3091, 1381, 2247, 378, 3056, 2912, 837, 2549, 2584, 2840, 1026, 171, 951, 711, 122, 1161, 457, 3510, 3467, 618, 166, 640, 3494, 559, 469, 2707, 866, 517, 728, 384, 2981, 61, 540, 3093, 203, 849, 2151, 796, 6, ...]"
1,1113822,Complete GST Course & Certification - Grow Your CA Practice,https://www.udemy.com/goods-and-services-tax/,True,75,2792,923,274,All Levels,39 hours,2017-03-09T16:34:20Z,Business Finance,209400,2017-03-09,16:34:20Z,2017,3,9,complet gst cours certif grow ca practic,"[2491, 1044, 452, 2981, 554, 391, 397, 2524, 630, 1365, 1190, 761, 646, 443, 3009, 2600, 116, 927, 2515, 2601, 1001, 2819, 2514, 2717, 2646, 2743, 432, 3363, 3291, 2543, 519, 577, 806, 259, 264, 2312, 1613, 3237, 2905, 3261, 262, 409, 748, 1168, 166, 569, 726, 3213, 2043, 2519, 2979, 3566, 1964, 1745, 3316, 1409, 39, 1019, 2925, 3442, 980, 2557, 2721, 1628, 40, 98, 1991, 2493, 2741, 2616, 307, 3663, 2630, 2121, 2679, 34, 411, 3231, 592, 3146, 2843, 2746, 352, 379, 1029, 326, 2667, 713, 2804, 1621, 532, 537, 459, 3334, 3415, 1366, 223, 902, 3423, 3294, ...]"
2,1006314,Financial Modeling for Business Analysts and Consultants,https://www.udemy.com/financial-modeling-for-business-analysts-and-consultants/,True,45,2174,74,51,Intermediate Level,2.5 hours,2016-12-19T19:26:30Z,Business Finance,97830,2016-12-19,19:26:30Z,2016,12,19,financi model busi analyst consult,"[247, 40, 121, 42, 824, 492, 142, 976, 151, 231, 808, 269, 548, 1027, 1028, 197, 380, 568, 831, 1197, 730, 825, 38, 1081, 319, 1077, 270, 943, 375, 392, 586, 321, 1090, 746, 923, 1162, 930, 924, 560, 944, 274, 1015, 191, 2996, 638, 292, 939, 456, 790, 56, 863, 2514, 994, 569, 420, 37, 1259, 179, 291, 785, 475, 1139, 1527, 3503, 1288, 834, 992, 82, 981, 101, 1708, 561, 587, 721, 687, 1410, 258, 3151, 1363, 3039, 2180, 925, 1117, 897, 780, 65, 634, 2874, 1421, 1590, 1160, 1791, 525, 180, 931, 701, 659, 425, 636, 373, ...]"
3,1210588,Beginner to Pro - Financial Analysis in Excel 2017,https://www.udemy.com/complete-excel-finance-course-from-beginner-to-pro/,True,95,2451,11,36,All Levels,3 hours,2017-05-30T20:07:24Z,Business Finance,232845,2017-05-30,20:07:24Z,2017,5,30,beginn pro financi analysi excel 2017,"[38, 60, 1099, 938, 545, 1185, 649, 132, 728, 741, 831, 1197, 1117, 40, 103, 2198, 939, 264, 259, 1139, 270, 449, 1643, 746, 943, 787, 170, 252, 3513, 1583, 45, 574, 1847, 1216, 612, 265, 846, 380, 984, 3279, 3112, 237, 723, 421, 765, 1362, 272, 395, 659, 2723, 254, 249, 1955, 39, 3019, 328, 83, 1154, 1840, 3327, 1013, 809, 1844, 1856, 209, 1017, 1843, 1829, 1584, 1966, 3458, 2464, 2724, 985, 2280, 933, 1836, 2159, 499, 2950, 403, 805, 302, 5, 475, 3396, 773, 189, 289, 890, 178, 25, 65, 1882, 454, 731, 90, 536, 992, 3315, ...]"
4,1011058,How To Maximize Your Profits Trading Options,https://www.udemy.com/how-to-maximize-your-profits-trading-options/,True,200,1276,45,26,Intermediate Level,2 hours,2016-12-13T14:57:18Z,Business Finance,255200,2016-12-13,14:57:18Z,2016,12,13,maxim profit trade option,"[295, 416, 188, 477, 234, 655, 217, 8, 410, 71, 1141, 222, 107, 738, 96, 43, 372, 49, 195, 51, 74, 510, 363, 97, 811, 99, 947, 798, 378, 1140, 236, 991, 315, 138, 152, 444, 829, 697, 330, 766, 59, 803, 514, 346, 1151, 1023

In [24]:
course_name = "Ultimate Investment Banking Course"
movie_index = df_cleaned[df_cleaned["course_title"]==course_name].index[0]
movie_index

0

In [27]:
df_cleaned.loc[movie_index,"Similarity Indices"]

array([  39,  242, 3479, ..., 1345, 1346, 3682], dtype=int64)

In [ ]:
df_cleaned.loc[movie_index,"Similarity Indices"]

In [36]:
for i in df_cleaned.loc[movie_index,"Similarity Indices"]:
    top_5_movie_indices = list(i[0:5])
    df_top_5_movies = df_cleaned.loc[top_5_movie_indices,"course_title"].values
df_top_5_movies

IndexError: invalid index to scalar variable.